In [1]:
!pip install pyspark arrow pyarrow

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import sys

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *


spark = SparkSession.builder \
    .master('local[4]')\
    .appName("Spark Streaming") \
    .config("spark.driver.extraJavaOptions", "-Dcom.sun.net.ssl.checkRevocation=false") \
    .config("spark.executor.extraJavaOptions", "-Dcom.sun.net.ssl.checkRevocation=false") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1") \
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true") \
    .config("spark.executor.instances", "1") \
    .config("spark.executor.memory", "512mb") \
    .getOrCreate()



Ivy Default Cache set to: /home/jupyter-olga_p/.ivy2/cache
The jars for the packages stored in: /home/jupyter-olga_p/.ivy2/jars
:: loading settings :: url = jar:file:/opt/tljh/user/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e89d01af-4369-4688-8551-3917461cae05;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.1 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found 

In [2]:
spark

In [35]:
def create_parquet_sink(df, file_name):
    return df \
            .writeStream \
            .format("parquet") \
            .option("path", "{f}".format(f=file_name)) \
            .option("checkpointLocation", "/user/jupyter-olga_p/{f}".format(f=file_name)) \
            .trigger(processingTime="5 seconds")

def kill_all():
    streams = SparkSession.builder.getOrCreate().streams.active
    if streams:
        for s in streams:
            desc = s.lastProgress["sources"][0]["description"]
            s.stop()
            print("Stopped {s}".format(s=desc))

In [36]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
import time


kafka_params = {
    "kafka.bootstrap.servers": "51.250.84.53:9092",
    "subscribe": "trades",
    "startingOffsets": """earliest""",
    "maxOffsetsPerTrigger": "5"
}

sdf = spark.readStream.format("kafka").options(**kafka_params).load()

In [26]:
sdf

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

# 
Приведите данные к правильная типам. Определите и удалите лишее.

In [27]:
parsed_sdf = sdf.select(
    F.to_date(F.col('timestamp')).alias('timestamp'), 
    F.get_json_object(F.col('value').cast(StringType()), '$.Price').cast(FloatType()).alias('price'))
parsed_sdf.printSchema()

root
 |-- timestamp: date (nullable = true)
 |-- price: float (nullable = true)



In [28]:
parsed_sdf

DataFrame[timestamp: date, price: float]

In [37]:
sink = create_parquet_sink(parsed_sdf, '/user/jupyter-olga_p/trades.parquet')
sink.start()
time.sleep(20)
kill_all()

Py4JJavaError: An error occurred while calling o323.start.
: java.io.IOException: mkdir of /user/jupyter-olga_p/trades.parquet/_spark_metadata failed
	at org.apache.hadoop.fs.FileSystem.primitiveMkdir(FileSystem.java:1067)
	at org.apache.hadoop.fs.DelegateToFileSystem.mkdir(DelegateToFileSystem.java:183)
	at org.apache.hadoop.fs.FilterFs.mkdir(FilterFs.java:201)
	at org.apache.hadoop.fs.FileContext$4.next(FileContext.java:730)
	at org.apache.hadoop.fs.FileContext$4.next(FileContext.java:726)
	at org.apache.hadoop.fs.FSLinkResolver.resolve(FSLinkResolver.java:90)
	at org.apache.hadoop.fs.FileContext.mkdir(FileContext.java:733)
	at org.apache.spark.sql.execution.streaming.FileContextBasedCheckpointFileManager.mkdirs(CheckpointFileManager.scala:303)
	at org.apache.spark.sql.execution.streaming.HDFSMetadataLog.<init>(HDFSMetadataLog.scala:66)
	at org.apache.spark.sql.execution.streaming.CompactibleFileStreamLog.<init>(CompactibleFileStreamLog.scala:47)
	at org.apache.spark.sql.execution.st

In [32]:
spark.read.parquet('/user/jupyter-olga_p/chk/trades.parquet').show()

AnalysisException: Path does not exist: file:/user/jupyter-olga_p/chk/trades.parquet;

In [38]:
!hdfs dfs -ls /user/jupyter-olga_p/

Found 4 items
drwxr-xr-x   - jupyter-olga_p hadoop          0 2022-05-24 18:06 /user/jupyter-olga_p/.sparkStaging
drwxr-xr-x   - jupyter-olga_p hadoop          0 2022-05-16 08:13 /user/jupyter-olga_p/Parshina_Olga.json
drwxr-xr-x   - jupyter-olga_p hadoop          0 2022-05-12 17:48 /user/jupyter-olga_p/file_stream
drwxr-xr-x   - jupyter-olga_p hadoop          0 2022-05-16 09:58 /user/jupyter-olga_p/wiki_file.json


In [18]:
!hdfs dfs -ls .

Found 4 items
drwxr-xr-x   - jupyter-olga_p hadoop          0 2022-05-24 18:06 .sparkStaging
drwxr-xr-x   - jupyter-olga_p hadoop          0 2022-05-16 08:13 Parshina_Olga.json
drwxr-xr-x   - jupyter-olga_p hadoop          0 2022-05-12 17:48 file_stream
drwxr-xr-x   - jupyter-olga_p hadoop          0 2022-05-16 09:58 wiki_file.json


In [32]:
from pyspark.sql.functions import rand

spark.conf.set("spark.sql.shuffle.partitions", "1")

impressions = (
  spark
    .readStream.format("rate").option("rowsPerSecond", "5").option("numPartitions", "1").load()
    .selectExpr("value AS adId", "timestamp AS impressionTime")
)

clicks = (
  spark
  .readStream.format("rate").option("rowsPerSecond", "5").option("numPartitions", "1").load()
  .where((rand() * 100).cast("integer") < 10)      # 10 out of every 100 impressions result in a click
  .selectExpr("(value - 50) AS adId ", "timestamp AS clickTime")      # -50 so that a click with same id as impression is generated later (i.e. delayed data).
  .where("adId > 0")
)

def create_console_sink(df):
    return df \
        .writeStream \
        .outputMode("append") \
        .format("console") \
        .trigger(processingTime="3 seconds") \
        .option("truncate", "false") \
        .option("numRows", "20")

def create_parquet_sink(df, file_name):
    return df \
            .writeStream \
            .format("parquet") \
            .option("path", "{f}".format(f=file_name)) \
            .option("checkpointLocation", "/home/jupyter-olga_p/HW/chk/{f}".format(f=file_name)) \
            .trigger(processingTime="10 seconds")

def kill_all():
    streams = SparkSession.builder.getOrCreate().streams.active
    if streams:
        for s in streams:
            desc = s.lastProgress["sources"][0]["description"]
            s.stop()
            print("Stopped {s}".format(s=desc))

In [31]:
!hdfs dfs -du -s -h /home/jupyter-olga_p/HW/chk/*

du: `/home/jupyter-olga_p/HW/chk/trades.parquet': No such file or directory


In [29]:
!pwd

/home/jupyter-olga_p/HW


In [13]:
sink = create_parquet_sink(parsed_sdf, 'trades.parquet')
sink.start()
time.sleep(20)
kill_all()

22/06/02 08:22:58 ERROR StreamMetadata: Error writing stream metadata StreamMetadata(1c86e578-5e06-41de-9949-42799c00fe85) to file:/user/jupyter-olga_p/trades.parquet/metadata
java.io.FileNotFoundException: File file:/user/jupyter-olga_p/trades.parquet does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:611)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:824)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.DelegateToFileSystem.getFileStatus(DelegateToFileSystem.java:126)
	at org.apache.hadoop.fs.DelegateToFileSystem.createInternal(DelegateToFileSystem.java:91)
	at org.apache.hadoop.fs.ChecksumFs$ChecksumFSOutputSummer.<init>(ChecksumFs.java:353)
	at org.apache.hadoop.fs.ChecksumFs.createInternal(ChecksumFs.java:400)
	at org.apache.hadoop.fs.AbstractFileSystem.create(AbstractFileSystem.java:596)
	at org.apache.hadoop.fs.Fil

Py4JJavaError: An error occurred while calling o181.start.
: java.io.FileNotFoundException: File file:/user/jupyter-olga_p/trades.parquet does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:611)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:824)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.DelegateToFileSystem.getFileStatus(DelegateToFileSystem.java:126)
	at org.apache.hadoop.fs.DelegateToFileSystem.createInternal(DelegateToFileSystem.java:91)
	at org.apache.hadoop.fs.ChecksumFs$ChecksumFSOutputSummer.<init>(ChecksumFs.java:353)
	at org.apache.hadoop.fs.ChecksumFs.createInternal(ChecksumFs.java:400)
	at org.apache.hadoop.fs.AbstractFileSystem.create(AbstractFileSystem.java:596)
	at org.apache.hadoop.fs.FileContext$3.next(FileContext.java:686)
	at org.apache.hadoop.fs.FileContext$3.next(FileContext.java:682)
	at org.apa

In [9]:
!hdfs dfs -du -s -h chk/*

du: `chk/trades.parquet': No such file or directory


In [35]:
s1 = create_parquet_sink(trades, 'trades.parquet')
# s2 = create_parquet_sink(clicks, 'clicks.parquet')
# s3 = create_parquet_sink(impressions.join(clicks, "adId"), 'inner.parquet')

NameError: name 'trades' is not defined

In [10]:
spark.read.parquet('trades.parquet').show()

AnalysisException: Unable to infer schema for Parquet at . It must be specified manually;